In [1]:
import streamlit as st
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
import google.generativeai as genai
from langchain.prompts import PromptTemplate



genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

/home/sumanpaudel/suman/Document-Bot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_splitter = RecursiveCharacterTextSplitter()
loader = PyPDFLoader('spark.pdf')
documents = loader.load()
# Split the text into chunks
texts = text_splitter.split_documents(documents)

In [3]:
# embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs = {'device': 'cpu'},
#     encode_kwargs = {'normalize_embeddings': False})

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [4]:
vector_store = FAISS.from_documents(texts, embeddings)

In [5]:
vector_store.save_local("faiss_index")

In [6]:
prompt_template = """
Answer the question as detailed as possible from the provided context, make sure to provide all the details,
if the answer is not in the provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
Context:\n {context}?\n
Question: \n{question}\n

Answer:
"""

# Initialize a ChatGoogleGenerativeAI model for conversational AI
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

# Create a prompt template with input variables "context" and "question"
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Load a question-answering chain with the specified model and prompt
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [7]:


# Load a FAISS vector database from a local file
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# Perform similarity search in the vector database based on the user question
docs = new_db.similarity_search("tell me actions in spark with examples")

# Obtain a conversational question-answering chain


# Use the conversational chain to get a response based on the user question and retrieved documents
response = chain(
    {"input_documents": docs, "question": "tell me actions in spark with examples"}, return_only_outputs=True
)

# Print the response to the console
print(response)

# Display the response in a Streamlit app (assuming 'st' is a Streamlit module)
# st.write("Reply: ", response["output_text"])

/home/sumanpaudel/suman/Document-Bot/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'output_text': 'answer is not available in the context'}
